In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stats import get_stats

from pathlib import Path

interval_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/interval')

for fp in interval_dir.glob('*.csv'):
    df = pd.read_csv(fp)
    df = df[df.State == 'ID']